## IRMA Scheduler Tutorial

Welcome to the Interactive Tutorial for the **IRMA (Installation Resource Management & Analytics) Scheduler**!

This notebook aims to guide new users/readers through the Mixed Integer Linear Programming (MILP) of the scheduler.

### Introduction
Imagine you are planning the schedule of installation activities for the deployment of an offshore system. 

📋 **Many Tasks**: Discrete work activities that need to be completed (e.g., anchor installation, cable laying)  
🚢 **Multiple Assets**: Resources (typically vessels) that can perform certain Tasks based on their capabilities (e.g., heavy-lift vessel, cable-laying vessel, support vessel)  
⚙️ **Task-Asset Assignments**: Multiple Assets can be assigned to each Task, where each assignment has an associated cost and time duration  
🌊 **Weather Windows**: Assets can only work in suitable sea conditions (e.g., wind speed limits, wave height limits)  
💰 **Time and Cost**: The schedule should aim to minimize the total cost of all tasks performed

How can the scheduler figure out what assets to assign to what tasks and when to perform each task to minimize time and/or cost?

### Mixed Integer Linear Programming (MILP)

A type of optimization problem that uses a mixture of integer, binary, and continuous variables subject to linear constraints to minimize an objective. 

As an example, let's say you own a truck delivery company with 3 trucks and you need to decide which trucks to send out for delivery, where each truck (x_i) has a cost of delivery and a time duration. The goal is to minimize cost of delivery, under a constraint that the total delivery time needs to be at least 12 hours. The only decisions are to either send out the truck for delivery (1) or not (0).

Minimize 
$$ 500x_1 + 400x_2 + 300x_3 $$

where

$$ 7x_1 + 6x_2 + 4x_3 \geq 12 $$

A MILP solver will realize that it needs at least two trucks for delivery and also figure out that Truck 2 ($x_2$) and Truck 3 ($x_3$) will not satisfy the constraint and neither will Truck 1 ($x_1$) and Truck 3 ($x_3$). That leaves the options of Truck 1 and Truck 2, or all three trucks. It will choose only Truck 1 and Truck 2 since that minimizes cost. (This also assumes that each truck can only be used once).

This tutorial only considers binary variables (for now).

$$x \in \{0,1\}$$

to determine what decisions to make to minimize the objective of the scheduler.

## Setup

First, let's import the necessary libraries and set up our environment.

In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os

# Add the FAModel path for imports
sys.path.append(os.path.join(os.getcwd(), 'famodel'))

# Import the IRMA scheduler
from famodel.irma.scheduler import Scheduler

# Set up plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = [12, 8]

print("✅ Libraries imported successfully!")
print("📊 Ready to explore the IRMA Scheduler!")

## Simple Case: Two Tasks, Two Assets

Let's start with the most basic scenario to understand the fundamentals:

### Tasks

Let's say that the installation of an offshore system requires two tasks: installing a mooring line, and installing an anchor, where each task has certain requirements that are needed to complete the task.

In [ ]:
tasks = [
    {
        'name': "install_mooring",
        'requirements': ['mooring_reel', 'positioning']
    },
    {
        'name': "install_anchor",
        'requirements': ['anchor_handling','positioning']
    }
]

# Display task information
task_df = pd.DataFrame(tasks)
print(task_df)

## Assets

And that there are two vessels (assets) that could potentially be used to perform these installations, each with their own set of capabilities, daily cost, and an integer value to represent what weather conditions it can operate in. For example, the Multi-Purpose Supply Vessel (MPSV) cannot operate in wave heights greater than 2 m, but the Anchor Handling Tug Supply Vessel (AHTS) can, but no greater than 4 m.

In [ ]:
# Define our installation vessel
assets = [
    {
        'name': 'AHTS', 
        'capabilities': ['anchor_handling', 'mooring_reel', 'positioning'],
        'daily_cost': 50000,
        'max_weather': 2
    },
    {
        'name': 'MPSV', 
        'capabilities': ['mooring_reel', 'positioning'],
        'daily_cost': 25000,
        'max_weather': 1
    }
]

print("🚢 Asset Definition:")
asset_df = pd.DataFrame(assets)
print(asset_df)

## The Task-Asset Matrix

Through a process that is still yet to be determined (TODO...Stein has something started), we can generate a **Task-Asset Matrix** that defines the cost and duration to perform each task by each set of assets.

Each row of the task-asset matrix represents a different task and each column of the task-asset matrix represents a combination of assets.

Entries with values of -1 represent task-asset pairs that are not feasible. Something like installing an anchor with a kayak.

In [ ]:
#                 |  AHTS    |  MPSV    | AHTS+MPSV |
# Install Mooring | (c, d)   | (c, d)   |   (c, d)  |
# Install Anchor  | (c, d)   | (c, d)   |   (c, d)  |
task_asset_matrix = np.array([
                  [(2000, 2), (1000, 3), (2500, 3)],
                  [(1500, 3), (-1,  -1), (4000, 2)]
])

print("🔗 Task-Asset Compatibility:")
print(task_asset_matrix)

## Asset Groups

Different combinations of assets can be used for each task, and each produce a different cost and duration to perform the task based on the capabilities of the assets and the requirements of the task.

The matrix generation process will filter out asset combinations that do not make sense (i.e., overlapping capabilities, maximum number of assets involved, extremely high costs, etc.)

In [ ]:
asset_groups = [
    {
        'assets': ['AHTS'], 
    },
    {
        'assets': ['MPSV'], 
    },
    {
        'assets': ['AHTS','MPSV'], 
    },
]

print("🚢 Asset Groups")
asset_group_df = pd.DataFrame(asset_groups)
print(asset_group_df)

## Time Periods & Weather

We can also define the planning horizon (timeline) as a set of time periods with given weather conditions. Time periods could be any duration of time (e.g., hours, days, weeks, etc.).

Good weather is normally designated by a 1, OK weather is designated by a 2, and bad weather is designated by a 3

In [ ]:
weather = [1, 1, 1, 1, 1]   # Start by defining 5 time periods, each with good weather

print("📅 Planning Horizon:")
weather_df = pd.DataFrame({
    'Weather_Condition': weather,
    'Description': ['Good weather'] * len(weather)
})
print(weather_df)

## Running the Simple Scheduler

Now let's create and run our first scheduler instance, which simply sets up many variables within the Scheduler class.

In [ ]:
# Create the scheduler for our simple scenario
print("🔧 Creating scheduler for simple scenario...")

scheduler = Scheduler(
    tasks=tasks,
    assets=assets,
    task_asset_matrix=task_asset_matrix,
    weather=weather,
    asset_groups=asset_groups,
    wordy=1  # Enable some debug output
)

print("✅ Scheduler created successfully!")

## Decision Variables

The initialization sets up the decision variables involved in the MILP optimization. They include:

- $X_{t,a} \in \{0,1\}$: 1 if task $t$ is assigned to asset group $a$, 0 otherwise
- $X_{t,p} \in \{0,1\}$: 1 if task $t$ is active in period $p$, 0 otherwise
- $X_{t,s} \in \{0,1\}$: 1 if task $t$ starts at period $s$, 0 otherwise

The only 'decisions' are whether certain asset groups are assigned to certain tasks $(X_{t,a})$, and when the task starts $(X_{t,s})$. The $X_{t,p}$ are also included to help organize the constraints in determining what periods each task occupy, based on the duration of the task-asset combination defined in the task-asset matrix.

The decision variable vector $x$ then follows the form of 

$$ x = [X_{t,a}, X_{t,p}, X_{t,s}] $$

where the length depends on the number of tasks $T$, the number of asset groups $A$, and the number of periods $P$

## Constraints

Now that the scheduler and decision variables initialized, we need to ensure the constraints are well defined to include in the MILP optimization.

In an MILP optimization, constraints are set to be linear and follow the form of 

$$
 A_{ub} \text{ } x \text{ } \leq b_{ub} \\
 A_{eq} \text{ } x \text{ } = b_{eq}    \\
 A_{lb} \text{ } x \text{ } \geq b_{lb} \\
$$

where $A$ and $b$ represent large vectors that when multipled by the binary decision variables of the $x$ vector, need to satisfy the constraint according to $b$.

In the example at the top of this page, we would structure the constraint as the following:

$$
\begin{bmatrix}
7 & 6 & 4
\end{bmatrix}
\begin{bmatrix}
x_1 \\ x_2 \\ x_3
\end{bmatrix}
\ge 12
$$

But now we need to develop many other constraints that fit this format and can be applied to our decision variables to ensure the problem solves how we want it to

### Constraint 1: An asset group can only be assigned to a task if the asset group can perform the task

Prevents Xta variables that correspond to invalid entries in the task-asset matrix from being turned on.

In [ ]:
# Solve the optimization problem
print("🚀 Solving the optimization problem...\n")

result_simple = scheduler_simple.solve()

if result_simple.success:
    print(f"\n🎉 Optimization successful!")
    print(f"💰 Total cost: {result_simple.fun:.0f}")
    
    # Decode and display the solution
    solution = scheduler_simple.decode_solution(result_simple.x)
    
    print("\n📋 Optimal Schedule:")
    for task_idx, assignment in solution['task_assignments'].items():
        task_name = tasks_simple[task_idx]['name']
        asset_idx = assignment['asset_group']  # In simple case, asset_group = asset index
        asset_name = assets_simple[asset_idx]['name']
        periods = assignment['periods']
        
        print(f"  📌 {task_name}:")
        print(f"     🚢 Assigned to: {asset_name}")
        print(f"     ⏰ Active in periods: {periods}")
        print(f"     ⏱️  Duration: {len(periods)} periods")
        
else:
    print(f"❌ Optimization failed: {result_simple.message}")